<a href="https://colab.research.google.com/github/MSumedhaa/Sentiment_Analysis/blob/main/sentimentanalysismajor_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install vaderSentiment

     |████████████████████████████████| 133kB 9.3MB/s 


In [ ]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [ ]:
vs= SentimentIntensityAnalyzer()

In [ ]:
#dataset

In [ ]:
import pandas as pd
df=pd.read_csv('/content/drive/My Drive/pythonseptember/ML/Restaurant_Reviews.tsv',sep='\t')

In [ ]:
df.head()

,Review,Liked
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
3,Stopped by during the late May bank holiday of...,1
4,The selection on the menu was great and so wer...,1


In [ ]:
df['label']=df['Liked'].apply(lambda x:'pos' if x>=1 else 'neg')

In [ ]:
df.head()

,Review,Liked,label
0,Wow... Loved this place.,1,pos
1,Crust is not good.,0,neg
2,Not tasty and the texture was just nasty.,0,neg
3,Stopped by during the late May bank holiday of...,1,pos
4,The selection on the menu was great and so wer...,1,pos


In [ ]:
df.shape

(1000, 3)

**CLEANING THE BAG OF WORDS**

In [ ]:
#preprocessing

In [ ]:
#lower
df.Review=df.Review.apply(lambda x:x.lower())

In [ ]:
df.head()

,Review,Liked,label
0,wow... loved this place.,1,pos
1,crust is not good.,0,neg
2,not tasty and the texture was just nasty.,0,neg
3,stopped by during the late may bank holiday of...,1,pos
4,the selection on the menu was great and so wer...,1,pos


In [ ]:
#punctuations
def remove_punctuations(text):
  for punctuation in string.puncuation:
    text=text.replace(punctuation,'')
  return text

In [ ]:
df.head()

,Review,Liked,label
0,wow... loved this place.,1,pos
1,crust is not good.,0,neg
2,not tasty and the texture was just nasty.,0,neg
3,stopped by during the late may bank holiday of...,1,pos
4,the selection on the menu was great and so wer...,1,pos


In [ ]:
df['label'].value_counts()

neg    500
pos    500
Name: label, dtype: int64

In [ ]:
a=df.iloc[0]['Review']

In [ ]:
c=df.iloc[1]['Review']

In [ ]:
vs.polarity_scores(c)

{'compound': -0.3412, 'neg': 0.445, 'neu': 0.555, 'pos': 0.0}

In [ ]:
df['compound_pred']=df['Review'].apply(lambda x:vs.polarity_scores(x)['compound'])

In [ ]:
df.head()

,Review,Liked,label,compound_pred
0,wow... loved this place.,1,pos,0.8271
1,crust is not good.,0,neg,-0.3412
2,not tasty and the texture was just nasty.,0,neg,-0.5574
3,stopped by during the late may bank holiday of...,1,pos,0.6908
4,the selection on the menu was great and so wer...,1,pos,0.6249


In [ ]:
#create another named col predicted sentiment
#if score>0,pos
#and if score<0,neg

In [ ]:
df[df['compound_pred']==0]

,Review,Liked,label,compound_pred
10,service was very prompt.,1,pos,0.0
11,would not go back.,0,neg,0.0
13,"i tried the cape cod ravoli, chicken, with cra...",1,pos,0.0
17,waitress was a little slow in service.,0,neg,0.0
24,so they performed.,1,pos,0.0
...,...,...,...,...
991,spend your money and time some place else.,0,neg,0.0
992,a lady at the table next to us found a live gr...,0,neg,0.0
995,i think food should have flavor and texture an...,0,neg,0.0
996,appetite instantly gone.,0,neg,0.0


In [ ]:
df['pred']=df['compound_pred'].apply(lambda x:'pos' if x>0 else 'neg')

In [ ]:
df.head()

,Review,Liked,label,compound_pred,pred
0,wow... loved this place.,1,pos,0.8271,pos
1,crust is not good.,0,neg,-0.3412,neg
2,not tasty and the texture was just nasty.,0,neg,-0.5574,neg
3,stopped by during the late may bank holiday of...,1,pos,0.6908,pos
4,the selection on the menu was great and so wer...,1,pos,0.6249,pos


In [ ]:
#evaluation
from sklearn.metrics import accuracy_score,confusion_matrix

In [ ]:
accuracy_score(df['label'],df['pred'])

0.815

In [ ]:
confusion_matrix(df['label'],df['pred'])

array([[404,  96],
       [ 89, 411]])

In [ ]:
#method 2-sklearn library

df

,Review,Liked,label,compound_pred,pred
0,wow... loved this place.,1,pos,0.8271,pos
1,crust is not good.,0,neg,-0.3412,neg
2,not tasty and the texture was just nasty.,0,neg,-0.5574,neg
3,stopped by during the late may bank holiday of...,1,pos,0.6908,pos
4,the selection on the menu was great and so wer...,1,pos,0.6249,pos
...,...,...,...,...,...
995,i think food should have flavor and texture an...,0,neg,0.0000,neg
996,appetite instantly gone.,0,neg,0.0000,neg
997,overall i was not impressed and would not go b...,0,neg,-0.3724,neg
998,"the whole experience was underwhelming, and i ...",0,neg,0.0000,neg


**CREATING THE  BAG OF WORDS MODEL**

In [ ]:
x=df.iloc[:,0].values#input
y=df.iloc[:,2].values#output

**SPLITTING THE DATASET INTO THE TRAINING SET AND TEST SET**

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,random_state=0)

In [ ]:
print(x_train.shape)
print(x_test.shape)

(750,)
(250,)


In [ ]:
import numpy as np
np.unique(y_train,return_counts=True)

(array(['neg', 'pos'], dtype=object), array([383, 367]))

In [ ]:
np.unique(y_test,return_counts=True)

(array(['neg', 'pos'], dtype=object), array([117, 133]))

**TRAING THE SVM MODEL ON THE TRAINING SET**

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline 

In [ ]:
final= Pipeline([('vect',TfidfVectorizer()),
                 ('model',SVC())])
#syntax for pipeline:pipeline([('method 1 var',method 1),
                            #  ('method 2 var',method 2)] )

In [ ]:
final.fit(x_train,y_train)

Pipeline(memory=None,
         steps=[('vect',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('model',
                 SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None,
                     coef0=0.0, decision_function_shape='ovr', de

**PREDICTING THE TEST RESULTS**

In [ ]:
y_pred=final.predict(x_test)

**ACCURACY  ACHIEVED**

In [ ]:
#evaluation
accuracy_score(y_pred,y_test)


0.82

**MAKING THE CONFUSION MATRIX**

In [ ]:
confusion_matrix(y_pred,y_test)

array([[100,  28],
       [ 17, 105]])

**streamlit web app**

In [ ]:
!pip install streamlit

     |████████████████████████████████| 7.4MB 8.9MB/s 
     |████████████████████████████████| 4.5MB 36.7MB/s 
     |████████████████████████████████| 163kB 48.4MB/s 
     |████████████████████████████████| 102kB 10.7MB/s 
     |████████████████████████████████| 133kB 43.5MB/s 
     |████████████████████████████████| 112kB 42.0MB/s 
     |████████████████████████████████| 6.8MB 38.9MB/s 
     |████████████████████████████████| 122kB 48.9MB/s 
     |████████████████████████████████| 71kB 9.1MB/s 
     |████████████████████████████████| 71kB 7.7MB/s 
  Created wheel for watchdog: filename=watchdog-0.10.3-cp36-none-any.whl size=73873 sha256=cdc2601fc7270df037fbc1f30e42836fd7ed8efb993392938d81ea27a81203b2
  Stored in directory: /root/.cache/pip/wheels/a8/1d/38/2c19bb311f67cc7b4d07a2ec5ea36ab1a0a0ea50db994a5bc7
  Created wheel for blinker: filename=blinker-1.4-cp36-none-any.whl size=13450 sha256=3d54bd664eff7e169bef3a7dd923bd164f98cc15e2c052f81b26758ab7d3bb4c
  Stored in directory: /root/.c

In [ ]:
!pip install pyngrok

In [ ]:
%%writefile app1.py
import streamlit as st
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidVectorizer
from sklearn.svm import SVC
df=pd.read_csv('/content/drive/My Drive/pythonseptember/ML/Restaurant_Reviews.tsv',sep='\t')
x=df.iloc[:,0].values#input
y=df.iloc[:,2].values#output
st.title("sentimental analysis")
st.subheader('TFIFD Vectorizer')
st.Write('this project is based on svc')
final= Pipeline([('vect',TfidfVectorizer()),
                 ('model',SVC())])
final.fit(x_train,y_train)
message= st.text_area("enter text","type here...")
op=final.predict([message])
if st.button("predict"):
  st.title(op)

In [ ]:
!streamlit run app1.py